In [1]:
from scipy.optimize import fmin_bfgs, fmin_l_bfgs_b
import csv
import numpy as np
import math
from collections import defaultdict
import operator
import hashlib
from datetime import datetime
import functools

In [2]:
with open('categories.txt', 'rb') as content_file:
    categories = content_file.read().split('\n')
#to make things faster i put the index where every category starts in a dictionary where the key are the product ids and 
#the values are the starting index of the category
catindex = defaultdict(int)
f = open('categories.txt','rb')
for i,l in enumerate(f):
    if len(l)-len(l.strip()) <=1:
        catindex[l.strip()]=i

#takes product id and returns category
def getCategory(pid, categories, catdict, depth):
    index = catdict[pid]
    value = set()
    j = index + 1
    while len(categories[j])-len(categories[j].strip())>1:
                ws = (categories[j].strip()).split(',')
                break
    return str((ws[:depth])[-1].strip())

In [3]:
def getYear(unix):
    time = datetime.utcfromtimestamp(float(unix))
    return time.strftime("%Y")

In [5]:
data = []
with open('100 core subset filtered.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['uid'], row['pid'], float(row['rating']), row['time']))

In [6]:
data[0]

('A1YXHALQFWN45C', '0783885008', 4.0, '1288310400')

In [7]:
def gethexdig(txt):
    c = hashlib.md5(txt.encode())
    return c.hexdigest()

gs= defaultdict(int)
for d in data:
    cat = getCategory(d[1], categories, catindex, 3)
    gs[cat]+=1
gs = set(sorted(gs.iteritems(), key= operator.itemgetter(1), reverse=True)[:50])
ghashed = set()
for g in gs:
    ghashed.add(gethexdig(g[0])) 

itemsuserrated = defaultdict(lambda: list())
useritems = defaultdict(lambda: list())
itemsuserratedwithtime = defaultdict(lambda: list())
for d in data:
    itemsuserrated[d[0]].append((d[1], d[3]))

for u in itemsuserrated:
    items = itemsuserrated[u]
    isorted = sorted(items, key = lambda tup: tup[1])
    itemsuserrated[u] = isorted
    useritems[u] = [i[0] for i in isorted]
    itemsuserratedwithtime[u] = [(i[0], i[1]) for i in isorted]

In [8]:
def same_year(u1,u2):
    d1= datetime.fromtimestamp(float(u1))
    d2= datetime.fromtimestamp(float(u2))
    delta = d1 - d2
    return delta.days <=365 and delta.days >= 0

def reviews_before_in_same_year(uid, pid):
    pids = []
    reviews = useritems[uid]
    reviewswithtime = itemsuserratedwithtime[uid]
    index = list(reviews).index(pid)
    year = reviewswithtime[index][1]
    for i in range(index):
        if same_year(year, reviewswithtime[i][1]):
            pids.append(reviewswithtime[i][0])
    return pids

# Every tuple in this dataset is as follows:
( user id, product id, rating score, current genre, previous genres ....)
user id: 0
product id: 1
rating score: 2
current genre: 3 
previous genres: 4-end

In [9]:
datag = []
depth = 3
dg = 'anything'
for d in data:
    index = list(useritems[d[0]]).index(d[1])
    cgenre = getCategory(d[1],categories,catindex,depth)
    if gethexdig(cgenre) not in ghashed:
        cgenre = dg
    cats = defaultdict(int)
    if not index==0:
        pids = reviews_before_in_same_year(d[0], d[1])
        for p in pids:
            cat = getCategory(p, categories, catindex, depth)
            if gethexdig(cat) not in ghashed :
                cat = dg
            cats[cat]+=1
    tup = (d[:3] + (cgenre,))
    for c in cats:
        tup = tup + (c, cats[c])
    datag.append(tup)

In [10]:
datag[0][0]

'A1YXHALQFWN45C'

In [11]:
users = {}
products = {}
us = set()
ps = set()
genres = defaultdict(int)
gs  = set()
i = 0
j = 0
k = 0
for d in datag:
    if d[0] not in us:
        users[d[0]] = i
        i+=1
        us.add(d[0])
    if d[1] not in ps:
        products[d[1]] = j
        j+=1
        ps.add(d[1])
    if d[3] not in gs:
        gs.add(d[3])
        genres[d[3]] = k
        k+=1

In [12]:
gside = len(genres)
gside

51

In [13]:
# datag = list(set(datag))
# # np.random.shuffle(datag)
# dtest = set()
# dvalid= set() #to choose best regualrization parameter and dimninsionality of factor vectors 
# dtestusers = defaultdict(int)
# dvalidusers = defaultdict(int)
# for d in datag:
#     cr = len(itemsuserrated[d[0]])
#     vc = int((cr * 20)/float(100))
#     if dtestusers[d[0]]< vc:
#         dtestusers[d[0]] +=1
#         dtest.add(d)
#     if d not in dtest and dvalidusers[d[0]]<vc:
#         dvalidusers[d[0]]+=1
#         dvalid.add(d)

# dtrain = set()
# for d in datag:
#     if d not in dtest and d not in dvalid:
#         dtrain.add(d)
# dtrain = list(dtrain)
# dtest = list(dtest)

In [14]:
x60 = int((len(datag) * 60) / float(100))
x20 = int((len(datag) * 20) / float(100))
dtrain = datag[:x60]
dvalid = datag[x60: x60+ x20]
end =  x60+ x20
dtest = datag[end:]
print len(dtrain), len(dvalid), len(dtest), len(datag)
print len(dtrain) + len(dvalid) + len(dtest) == len(datag)

40227 13409 13410 67046
True


In [15]:
dtrain[0][0]

'A1YXHALQFWN45C'

In [16]:
us = set()
ps = set()
gs = set()
for d in datag:
    us.add(d[0])
    ps.add(d[1])
    for i in range(4,len(d),2):
        gs.add((d[i], d[3]))

In [17]:
uintrain = set()
pintrain = set()
gintrain = set()
for d in dtrain:
    pintrain.add(d[1])
    uintrain.add(d[0])
    for i in xrange(4, len(d),2):
        gintrain.add((d[i], d[3]))

In [18]:
len(gintrain), len(gs)

(2601, 2601)

In [19]:
def getPredictions(params,k, data):
    alphao = params[0]
    betauo = params[1: len(users)+1]
    end = len(users)+1
    betaio = params[end: end+len(products)]
    end = end+len(products)
    bij = np.array(params[end: end+(gside**2)])
    betaggo = bij.reshape(gside, gside)
    end = end + (gside**2)
    gss = np.array(params[end:])
    grr = gss.reshape(len(users)+len(products),k)
    gammuo = grr[:len(users),:]
    gammio = grr[len(users):,:]
    predictions = []
    for d in data:
        pred = alphao + betauo[users[d[0]]] + betaio[products[d[1]]] + (gammuo[users[d[0]]].T.dot(gammio[products[d[1]]]))
        for i in xrange(4, len(d), 2):
            pred += betaggo[genres[d[i]]][genres[d[3]]] * np.log10(d[i+1])
        predictions.append(pred)
    return predictions

In [20]:
def mse(params,k, data):
    s = 0
    predictions = getPredictions(params, k, data)
    for i, d in enumerate(data):
        s+= (predictions[i] - d[2])**2
    s = s
    return s / float(len(data))

In [21]:
def mean(data):
    s = 0
    for d in data:
        s+= d[2]
    return  s/ float(len(data))

In [22]:
def variance(data):
    m = mean(data)
    s = 0
    for d in data:
        s += (m - d[2]) ** 2
    return s/float(len(data))

In [23]:
def rmse(params,k,data):
    predictions = getPredictions(params, k, data)
    s = 0
    for i, d in enumerate(data):
        s+= (predictions[i] - d[2])**2
    return math.sqrt(s / float(len(data)))

In [24]:
def Rsquare_statistic(params,k, data):
    predictions = getPredictions(params,k, data)
    error = mse(params,k, data)
    var = variance(data)
    return error/float(var)

In [25]:
def compute_cost(initial,lam1, lam2, k, data):
    alpha = initial[0]
    betau = initial[1:len(users) + 1]
    end = len(users)+1
    betai = initial[end:end + len(products)]
    end = end + len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betagg = bij.reshape(gside, gside)
    end = end + (gside**2)
    gss = np.array(initial[end:])
    gr = gss.reshape(len(users)+len(products),k)
    gammu = gr[:len(users),:]
    gammi = gr[len(users):,:]
    J = 0
    for d in data:
        term = alpha + betau[users[d[0]]] + betai[products[d[1]]] +(gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2]
        for i in xrange(4, len(d), 2):
            term +=  betagg[genres[d[i]]][genres[d[3]]]* np.log10(d[i+1])
        J+= term ** 2
    for u in us:
        J += lam1 * (np.square(betau[users[u]])+ np.sum(np.square(gammu[users[u]])))
    for p in ps:
        J+= lam1 * ( np.square(betai[products[p]]) + np.sum(np.square(gammi[products[p]])))
    for g in gintrain:
        J+= lam2 * np.square(betagg[genres[g[0]]][genres[g[1]]])
    gr = compute_gradient(alpha, betau, betai, betagg, gammu, gammi,lam1, lam2, k, data)
    return J, gr.flatten()

In [26]:
def compute_gradient(alpha,betau, betai,betagg, gammu, gammi,lam1, lam2, k, data):
    ga = 0
    gbu = np.zeros(len(users))
    gbi = np.zeros(len(products))
    ggu = np.zeros((len(users),k))
    ggi = np.zeros((len(products),k))
    gbgg = np.zeros((gside, gside))
    for d in data:
        term = alpha + betau[users[d[0]]] + betai[products[d[1]]] +(gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2]
        for i in xrange(4, len(d),2):
            term +=  betagg[genres[d[i]]][genres[d[3]]] * np.log10(d[i+1])
        ga+= 2 * term
        gbu[users[d[0]]] += 2 * term
        gbi[products[d[1]]] += 2 * term
        ggu[users[d[0]]] += 2 * gammi[products[d[1]]] * term
        ggi[products[d[1]]] += 2 * gammu[users[d[0]]] * term
        for i in xrange(4, len(d),2):
            gbgg[genres[d[i]]][genres[d[3]]] += 2* term * np.log10(d[i+1])
    for u in us:
        gbu[users[u]] += 2 * lam1 * betau[users[u]]
        ggu[users[u]] += 2 * lam1 * gammu[users[u]]
    for p in ps:
        gbi[products[p]] += 2 * lam1 * betai[products[p]]
        ggi[products[p]] += 2 * lam1 * gammi[products[p]]
    for g in gintrain:
        gbgg[genres[g[0]]][genres[g[1]]] += 2 * lam2 * betagg[genres[g[0]]][genres[g[1]]]
    return np.array([ga] + list(gbu) + list(gbi) + list(gbgg.flatten()) + list(ggu.flatten())+ list(ggi.flatten()))

In [30]:
def cost(initial, k, data):
    alpha = initial[0]
    betau = initial[1:len(users) + 1]
    end = len(users)+1
    betai = initial[end:end + len(products)]
    end = end + len(products)
    bij = np.array(initial[end: end+(gside**2)])
    betagg = bij.reshape(gside, gside)
    end = end + (gside**2)
    gss = np.array(initial[end:])
    gr = gss.reshape(len(users)+len(products),k)
    gammu = gr[:len(users),:]
    gammi = gr[len(users):,:]
    J = 0
    for d in data:
        term = alpha + betau[users[d[0]]] + betai[products[d[1]]] +(gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2]
        for i in xrange(4, len(d), 2):
            term +=  betagg[genres[d[i]]][genres[d[3]]]* np.log10(d[i+1])
        J+= term ** 2

    return J

In [31]:
# class ConvergedException(Exception):
#     pass
# def check(xk):
#     global result
#     global minmse
#     global count
#     global presult
#     global nit 
    
#     nit +=1
# #     print nit

#     cmse = mse(xk,k, dvalid)
    
#     if cmse > minmse:
#         if nit >= 100:
#             count +=1
#             if count == 5:
#                 print 'here'
#                 result[:] = presult
#                 raise ConvergedException()
#                 return

#     else:
#         minmse = cmse
#         presult[:] = xk
#         count = 0
# #     print cmse, minmse, presult[0]
#     return
    

In [32]:
# class ConvergedException(Exception):
#     pass
# def check_updated(xk):
#     global result
#     global presult
#     global count
#     global nit
#     global pmse
    
#     count+=1
#     nit+=1
#     if count==20:
#         count = 0
#         cmse = mse(xk,k, dvalid)
# #         print ccost
#         if cmse> pmse:
#             print 'here'
#             result[:] = presult
#             raise ConvergedException()
#             return
#         else:
#             pmse = cmse
#             presult[:] = xk    

In [33]:
k = 10
s = 0
for d in data:
    s+= d[2]
alpha = s / float(len(data))
betau = np.random.normal(0,0.1, len(users))
betai = np.random.normal(0,0.1, len(products))
betagg = np.random.normal(0,0.1, (gside, gside))
for u in users:
    if u not in uintrain:
        betau[users[u]] = 0
for p in products:
    if p not in pintrain:
        betai[products[p]] = 0
for g in gs:
    if g not in gintrain:
        betagg[genres[g[0]]][genres[g[1]]] = 0
gammu = np.random.normal(0,0.1, (len(users), k)) 
gammi = np.random.normal(0,0.1, (len(products), k)) 
for u in users:
    if u not in uintrain:
        gammu[users[u]] = 0
for p in products:
    if p not in pintrain:
        gammi[products[p]] = 0
initial = [alpha] + list(betau) + list(betai) + list(betagg.flatten()) + list(gammu.flatten()) + list(gammi.flatten())

In [34]:
class ConvergedException(Exception):
    pass
def check_updated(xk):
    global result
    global minmse
    global count
    global presult
    global nit 
    
    nit +=1
#     print nit

#     cmse = mse(xk,k, dvalid)
    cmse = cost(xk, k , dvalid)
    if cmse > minmse:
            count +=1
            if count == 30:
                print 'here'
                result[:] = presult
                raise ConvergedException()
                return

    else:
        minmse = cmse
        presult[:] = xk
        count = 0
#     print cmse, minmse, presult[0]
    return
    

In [ ]:
lams1 = [10]
lams2 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000,10000, 100000]

for l1 in lams1:
    for l2 in lams2:
        presult = []
        count = 0
        nit = 0
        minmse = 100000000000
        result = []
        pms = []
        try:
            pms, c,d = fmin_l_bfgs_b(compute_cost, x0=initial,args=(l1,l2, k, dtrain, ), disp = 0, callback = check_updated)
            pms = presult
        except ConvergedException:
            pms = result
        print 'converged after: ' + str(nit)
        print "lambda "+ '"' +  str(l1) + '" -> "' + str(l2) + '" : ' + ' '  + str(cost(pms, k, dvalid)) + ' ' + str(cost(pms,k, dtest)) + ' ' +  str(mse(pms, k, dvalid)) + ' ' + str(mse(pms,k, dtest))

here
converged after: 131
lambda "10" -> "0" :  10582.7388705 10330.9586175 0.789226554593 0.770392141502
here
converged after: 135
lambda "10" -> "0.0001" :  10584.066221 10334.212154 0.789325544115 0.77063476167
here
converged after: 129
lambda "10" -> "0.001" :  10572.3454566 10328.731895 0.788451447281 0.770226092092
here
converged after: 131
lambda "10" -> "0.01" :  10574.1059799 10322.6866711 0.788582741435 0.769775292403
here
converged after: 130
lambda "10" -> "0.1" :  10575.5493386 10327.1976437 0.788690382477 0.770111681108

In [ ]:
import matplotlib
from pylab import *
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
lams = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
x = lams
y = [ 0.789226554593 , 0.789325544115, 0.788451447281, 0.788582741435 , 0.788690382477 
    ]
# color = ['blue'] * 7 + ['red'] + ['blue'] * 3
plt.scatter(lams[7], y[7], color = 'black', s = 100, marker="o")
plt.semilogx(x, y, color = 'red')
# legend = plt.legend(loc='upper right', shadow=True, fontsize='large')
plt.xlabel('Lambda Value For Adjacent Genre Bias Terms')
plt.ylabel('mse on Validation Set')
plt.title("One Year Memory Pairwise Genre Bias Terms Model Regularization Parameter vs. mse")
plt.show()

In [59]:
global result
lams1 = [10]
lams2 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
# lams2 = [1000000]
with open('Regression baseline - 1 year memory log #times 100 core.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=['lambda1','lambda2','k','value'])
    writer.writeheader()
    for l1 in lams1:
        for l2 in lams2: 
            minmse = mse(initialg,k,dvalid)
            presult = []
            params = []
            count = 0
            result = []
            nit = 0
            try:
                params, c,d = fmin_l_bfgs_b(compute_cost, x0=initialg,args=(l1, l2 ,k, dtrain, ), disp = 0, callback = check)
            except ConvergedException:
                params = result
            print 'converged'
            print "lambda "+ '"' +  str(l1) + '" -> "' + str(l2) + '" : ' + ' ' + str(mse(params, k, dtest))
            for i, v in enumerate(params):
                writer.writerow({'lambda1': l1,'lambda2': l2, 'k': k, 'value': v})   

here
converged
lambda "10" -> "0" :  0.769601178227
here
converged
lambda "10" -> "0.0001" :  0.769893885363
here
converged
lambda "10" -> "0.001" :  0.770128099183
here
converged
lambda "10" -> "0.01" :  0.770409379957
here
converged
lambda "10" -> "0.1" :  0.769707303199
here
converged
lambda "10" -> "1" :  0.767213764461
here
converged
lambda "10" -> "10" :  0.757037253983
here
converged
lambda "10" -> "100" :  0.751501591701
here
converged
lambda "10" -> "1000" :  0.750998406342
here
converged
lambda "10" -> "10000" :  0.754332050866


In [60]:
params = defaultdict(lambda: defaultdict(list))
with open('Regression baseline - 1 year memory log #times 100 core.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        params[(float(row['lambda1']), float(row['lambda2']))][int(row['k'])].append(float(row['value']))

In [61]:
op = []
minCost = 1000000
predictions = []
for l in params:
    for k in params[l]:
        c= mse(params[l][k], k ,dvalid)
#             print l1, k, c
        if c < minCost:
            minCost = c
            minlambda = l
            mink = k
            op = params[l][k]
mink, minlambda

(10, (10.0, 1000.0))

In [64]:
# with open("1 Year Memory Model Optimized Parameters.csv", "wb") as f:
#     writer = csv.writer(f)
#     for o in op:
#         writer.writerow([o])

In [65]:
op = []
with open("1 Year Memory Model Optimized Parameters.csv", 'rb') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        op.append(float(row[0]))

In [66]:
print "mse: " + str(mse(op,k,dtest))
# print "R Squared Statistic: " + str( mse(op, k, dtest) /float(variance(data)))

mse: 0.750998406342


In [67]:
mse(op, k, dtrain)

0.60351925007068252

In [36]:
usergenres = defaultdict(lambda: set())
for u in useritems:
    for i in useritems[u]:
        cat = getCategory(i, categories, catindex, 3)
        if gethexdig(cat) not in ghashed:
            cat = 'Anything'
        usergenres[u].add(cat)

In [92]:
k = 10
perfu = defaultdict(int)
for j in range(len(genres)):
    diverse = [ u for u in usergenres if len(usergenres[u]) > j]
#     print len(diverse), len(dtest)
    dlist = []
#     ndlist = []
    for d in dtest:
        if d[0] in diverse:
            dlist.append(d)
#     print len(dlist)

#         else: 
#             ndlist.append(d)
    if len(dlist) > 0:
        perfu[j] = mse(op, k , dlist)

In [178]:
x = [p for p in perfu]
y = [perfu[p] for p in perfu]
yb = [0.768155048088 for i in range(len(perf))]
# for p in perf:
plt.plot(x, y, color = 'red', label ='1 Year Memory (log #times)')
plt.plot(x, yb, color = 'blue', label= 'Baseline')
plt.xlabel('Number of Genres Read')
plt.ylabel('mse')
ylim(0,0.9)
legend = plt.legend(loc='upper right', shadow=True, fontsize='large')
plt.title('Performance of Model on Users Who Read N Number of Genres')
plt.show()


In [90]:
k = 10
perf = defaultdict(float)
catlist = defaultdict(list)
for j in range(len(genres)):
    glist = []
    for d in dtest:
        cat = getCategory(d[1], categories, catindex, 3)
        if gethexdig(cat) not in ghashed:
            cat = 'Anything'
        catlist[cat].append(d)
for c in catlist:
    perf[c] = mse(op, k, catlist[c])

In [179]:
labels = [p for p in perf]
y = [perf[p] for p in perf]
x = range(len(labels))
yb = [0.768155048088 for i in range(len(perf))]
# for p in perf:
plt.plot(x, y, color = 'red', label ='1 Year Memory (log #times)')
plt.xticks(x, labels, rotation='vertical')

plt.plot(x, yb, color = 'blue', label= 'Baseline')
plt.xlabel('Genre of Item')
plt.ylabel('mse')
legend = plt.legend(loc='upper right', shadow=True, fontsize='large')
plt.title('Performance of Model According to Genre of Item')
plt.show()

In [1]:
import matplotlib
from pylab import *
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
lams2 = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

x = lams2
y = [0.781125413464, 0.782763031305, 0.7809551286, 0.782318380065, 0.781982051293, 0.779774698916, 0.770221247767, 0.765388004321,
     0.809337339406]
# color = ['blue'] * 7 + ['red'] + ['blue'] * 3
plt.scatter(lams2[7], y[7], color = 'black', s = 100, marker="o")
plt.semilogx(x, y, color = 'red')
# legend = plt.legend(loc='upper right', shadow=True, fontsize='large')
plt.xlabel('Lambda Value For the 1 Year Memory Pairwise Genre Bias Terms')
plt.title("1 Year Memory Pairwise Genre Bias Terms Model Regularization Graph")
plt.ylabel('mse on Validation Set')
plt.show()

In [ ]:
cpersorted = [ c[0] for c in sorted(perf.iteritems(), key = operator.itemgetter(1))[:5]]

usergenres = defaultdict(lambda: set())
userreviews = defaultdict(int)
for u in useritems:
    for i in useritems[u]:
        cat = getCategory(i, categories, catindex, 3)
        if gethexdig(cat) not in ghashed:
            cat = 'Anything'
        if cat in cpersorted:
            usergenres[u].add(cat)
        userreviews[u]+=1

In [ ]:
m = 0
for u in useritems:
    if len(useritems[u]) > m:
        m = len(useritems[u])

In [213]:
k = 10
perfu = defaultdict(int)
for j in range(len(genres)):
    diverse = [ u for u in usergenres if len(usergenres[u]) > j]
#     print len(diverse), len(dtest)
    dlist = []
#     ndlist = []
    for d in dtest:
        if d[0] in diverse and getCategory(d[1], categories, catindex, 3) in cpersorted:
            dlist.append(d)
#     print len(dlist)

#         else: 
#             ndlist.append(d)
    if len(dlist) > 0:
        perfu[j] = mse(op, k , dlist)

In [214]:
x = [p for p in perfu]
y = [perfu[p] for p in perfu]
yb = [0.768155048088 for i in range(len(perfu))]
# for p in perf:
plt.plot(x, y, color = 'red', label ='1 Year Memory (log #times)')
plt.plot(x, yb, color = 'blue', label= 'Baseline')
plt.xlabel('Number of Genres Read')
plt.ylabel('mse')
ylim(0,0.9)
legend = plt.legend(loc='upper right', shadow=True, fontsize='large')
plt.title('Performance of Model on Users Who Read N Number of Genres')
plt.show()


KeyboardInterrupt: 

In [121]:
ls = [g for g in gpop] 

In [118]:
gpop = defaultdict(float)
for d in dtrain:
    cat = getCategory(d[1], categories, catindex, 3)
    if gethexdig(cat) not in ghashed:
        cat = 'Anything'
    gpop[cat] +=1
for g in gpop:
    gpop[g] = gpop[g] / float(len(dtrain))


In [183]:
labelsper= [p for p in perf]
# labelspop =[p for p in gpop]
yper = []
ypop = []
for p in perf:
    yper.append(perf[p])
    ypop.append(gpop[p])

    
# y = [perf[p] for p in perf]
x = range(len(labelsper))
yb = [0.768155048088 for i in range(len(perf))]
# for p in perf:
plt.bar(x, yper, color = 'red', label ='Performance According To Genre')
plt.xticks(x, labels, rotation='vertical')

plt.bar(x, ypop, color = 'blue', label= 'Genre Popularity in Training set')
plt.xlabel('Genre of Item')
plt.ylabel('mse / genre popularity')
legend = plt.legend(loc='upper right', shadow=True, fontsize='large')
plt.title('Genre Performance Vs. Genre Popularity')
plt.show()

In [186]:
y = np.matrix([yper, ypop])
y = y.reshape((51, 2))
df2 = pd.DataFrame(y, columns= ['Genre mse', 'Genre Popularity'], index = labelsper)
df2.plot.bar()
plt.xlabel('Genre of Item')
plt.ylabel('mse / genre popularity')
plt.title('Genre mse Vs. Genre Popularity in Training Set')
plt.show()

In [70]:
def computeNumericalGradient(params,grad, lam,k, data):
    e = 1e-4
    numgrads = np.zeros(len(params))
    pp = np.zeros(len(params))
    for i,p in enumerate(params):
        pp[i] = e
        loss1,_ = compute_cost(params-pp, lam, k,data)
        loss2,_ = compute_cost(params+pp, lam,k, data)
#         print loss1, loss2
        numgrads[i] = (loss2 - loss1) / float(2*e)
        pp[i] = 0
        print round(numgrads[i], 3), round(grad[i], 3)
#         print abs(numgrads[i]-grad[i])
    return numgrads

In [157]:
# alpha = s / float(len(data))
# betau = np.random.normal(0,0.1, len(users))
# betai = np.random.normal(0,0.1, len(products))
# betagg = np.random.normal(0,0.1, (gside, gside))
# initial = [alpha] + list(betau) + list(betai) + list(betagg.flatten())
# gammu = np.random.normal(0,0.1, (len(users), k)) 
# gammi = np.random.normal(0,0.1, (len(products), k)) 
# initial += list(gammu.flatten()) + list(gammi.flatten())
# k = 1
# g = compute_gradient(alpha, betau, betai,betagg, gammu, gammi, 0, k, dtrain)
# computeNumericalGradient(initial,g , 0, k , dtrain )
# # gammu

In [48]:
params = op
k = mink
alphao = params[0]
betauo = params[1: len(users)+1]
end = len(users)+1
betaio = params[end: end+len(products)]
end = end+len(products)
bij = np.array(params[end: end+(gside**2)])
betaggo = bij.reshape(gside, gside)
end = end + (gside**2)
gs = np.array(params[end:])
gr = gs.reshape(len(users)+len(products),k)
gammuo = gr[:len(users),:]
gammio = gr[len(users):,:]

In [49]:
def get_genre_from_id(i):
    for g in genres:
        if genres[g] == i:
            return g

In [50]:
ggbias = defaultdict(int)
ggbiasd = defaultdict(lambda: defaultdict(int))
for i in range(len(betaggo)):
    for j in range(len(betaggo[i])):
        g1 = get_genre_from_id(i)
        g2 = get_genre_from_id(j)
        ggbias[(g1,g2)] = betaggo[i][j]
        ggbiasd[g1][g2] = betaggo[i][j]

In [51]:
# sorted(ggbias.iteritems(), key=operator.itemgetter(1), reverse=True)

In [53]:
import pandas as pd
frame = pd.DataFrame.from_dict(data=ggbiasd, dtype=float)
frame = frame.fillna(value=0)

import seaborn as sns
import matplotlib.pyplot as plt

mask = np.zeros_like(frame, dtype=np.bool)
for i in range(frame.shape[0]):
    for j in range(frame.shape[1]):
        if frame.columns[j] == frame.index[i]:
            mask[i,j] = True

with sns.axes_style('whitegrid'):
    ax = sns.heatmap(frame, mask=mask,center= 0,  robust=True,linewidths=0.5)
plt.yticks(rotation=0) 
plt.xticks(rotation=90)
fig = plt.gcf()
plt.title('Adjacent Migrations - depth: '+ str(depth))
fig.subplots_adjust(bottom=0.3, left = 0.2)
plt.show()

In [165]:
fig.savefig("From Model 1 Year Memory Migrations.pdf")